In [1]:
# Cell 2 — imports and setup
import os
import pandas as pd
from supabase import create_client
from dotenv import load_dotenv

load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)


In [2]:
# Cell 3 — pull table from Supabase
response = (
    supabase
    .table("background_check_time_series")
    .select("*")
    .execute()
)

df = pd.DataFrame(response.data)


In [3]:
# Cell 4 — basic sanity checks
df.head()


,state,state_abb,fips,year,year_frac,law_class,law_class_subtype,handguns_or_long_guns,age_for_minimum_age_laws,law_id
0,Alaska,AK,2,1979,1,Castle doctrine,N/A,handgun and long gun,#N/A,AK1005
1,Alaska,AK,2,1980,1,Castle doctrine,N/A,handgun and long gun,#N/A,AK1005
2,Alaska,AK,2,1981,1,Castle doctrine,N/A,handgun and long gun,#N/A,AK1005
3,Alaska,AK,2,1982,1,Castle doctrine,N/A,handgun and long gun,#N/A,AK1005
4,Alaska,AK,2,1983,1,Castle doctrine,N/A,handgun and long gun,#N/A,AK1005


In [4]:
import plotly.express as px

# ensure year is int
df["year"] = df["year"].astype(int)

# find first year each law appears
law_start = (
    df.groupby("law_id", as_index=False)["year"]
      .min()
      .rename(columns={"year": "start_year"})
)

# count how many laws exist by each year
years = sorted(df["year"].unique())

df_cum = []
for y in years:
    df_cum.append({
        "year": y,
        "law_count": (law_start["start_year"] <= y).sum()
    })

df_cum = pd.DataFrame(df_cum)


In [5]:
fig = px.line(
    df_cum,
    x="year",
    y="law_count",
    animation_frame="year",
    markers=True,
    title="Cumulative Number of Gun Laws Over Time"
)

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Cumulative Laws",
    transition={"duration": 300}
)

fig.show()
